# Vehicle Activity Dashboard

This notebook will demonstrate how to make a vehicle activity dashboard that shows the vehicle's location, speed and direction through a whole trip.

In [57]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, gridplot,output_notebook, show
from bokeh.models import Range1d, HoverTool,Line, Circle, FixedTicker, BoxZoomTool,DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, ResetTool,Title,ColumnDataSource
from bokeh.tile_providers import CARTODBPOSITRON_RETINA
output_notebook()

Loading BokehJS ...

## Data Preparation
Before we make any plots we need to transform some variables into appropriate format. Let's specify some plotting configurations first.

Plot every n datapoints where n = sampling.

In [53]:
sampling = 10

If offset is True, the r axis starts at the Start Time of the trip rather than the Gentime.

In [54]:
offset = True

Specify the primary keys of the trip we want to plot.

In [56]:
RxDevice = 6565
FileID = 7716380
TxDevice = 6565

Some configs for future plotting.

In [58]:
basic_tools = "box_select,pan,wheel_zoom,reset,box_zoom"
radial_tick_scales = [1, 2, 5, 10, 20, 60, 120, 240, 480, 1000, 2000, 5000]
angular_tick_sep = 15

Read in data.

In [59]:
df = pd.read_csv('TripStart_41728_p038.csv',header=None, usecols = [0,1,2,3,7,8,10,11,15], names = ["RX_DEVICE", "FILE_ID", "TX_DEVICE", "GENTIME", "LATITUDE", "LONGITUDE", "SPEED", "HEADING", "YAW_RATE"])
df.head()

,RX_DEVICE,FILE_ID,TX_DEVICE,GENTIME,LATITUDE,LONGITUDE,SPEED,HEADING,YAW_RATE
0,6512,7125966,6512,323254442760265,43.437309,-83.939384,31.459999,104.9250,0.2
1,6512,7125966,6512,323254442860111,43.437302,-83.939346,31.540001,105.0125,0.2
2,6512,7125966,6512,323254442960170,43.437294,-83.939308,31.480000,104.5750,0.0
3,6512,7125966,6512,323254443060049,43.437290,-83.939270,31.500000,104.5500,0.0
4,6512,7125966,6512,323254443160143,43.437283,-83.939232,31.540001,104.5250,0.0


Select the trip.

In [60]:
df = df[(df['RX_DEVICE'] == RxDevice) & (df['FILE_ID']==FileID)&(df['TX_DEVICE']==TxDevice)]

In [67]:
len(df)

4959

In [68]:
sample_df = df.iloc[::sampling,:].reset_index(drop=True)

Convert GENTIME from "number of **microseconds** since Jan 1, 2004" to number of **minutes** since trip start.

In [69]:
sample_df["GENTIME"] = sample_df["GENTIME"]/(6*10**7)

if offset:
    sample_df["GENTIME"] = sample_df["GENTIME"]-sample_df["GENTIME"].iloc[0]

In [70]:
sample_df.head()

,RX_DEVICE,FILE_ID,TX_DEVICE,GENTIME,LATITUDE,LONGITUDE,SPEED,HEADING,YAW_RATE
0,6565,7716380,6565,0.000000,42.518993,-83.743454,0.04,203.625,0.000000
1,6565,7716380,6565,0.016661,42.519043,-83.743431,1.24,203.625,11.610000
2,6565,7716380,6565,0.033328,42.519054,-83.743416,1.24,203.625,16.709999
3,6565,7716380,6565,0.050004,42.519062,-83.743408,0.82,203.625,16.740000
4,6565,7716380,6565,0.066660,42.519062,-83.743401,0.28,203.625,1.950000


Convert speed unit meter/second to mph.

In [71]:
sample_df["SPEED"] = sample_df["SPEED"]*2.23694

Create new variables for radial plot that shows a vehicle's direction at a certain point.

In [73]:
sample_df['x'] = sample_df["GENTIME"]*np.cos((360.0-sample_df["HEADING"]+90.0)*np.pi/180.0)
sample_df['y'] = sample_df["GENTIME"]*np.sin((360.0-sample_df["HEADING"]+90.0)*np.pi/180.0)
sample_df = sample_df.drop(["RX_DEVICE", "TX_DEVICE", "FILE_ID"], axis=1)
sample_df = sample_df.sort_values(["GENTIME"])

In [74]:
sample_df.head()

,GENTIME,LATITUDE,LONGITUDE,SPEED,HEADING,YAW_RATE,x,y
0,0.000000,42.518993,-83.743454,0.089478,203.625,0.000000,-0.000000,-0.000000
1,0.016661,42.519043,-83.743431,2.773806,203.625,11.610000,-0.006677,-0.015265
2,0.033328,42.519054,-83.743416,2.773806,203.625,16.709999,-0.013356,-0.030534
3,0.050004,42.519062,-83.743408,1.834291,203.625,16.740000,-0.020039,-0.045813
4,0.066660,42.519062,-83.743401,0.626343,203.625,1.950000,-0.026714,-0.061073


We will need to project latitude, longitude degree coordinates to meters in web mercator projection so that we can show the path on custom map tiles.

In [75]:
def latlng_to_meters(df, lat_name, lng_name):
    df = df.copy()
    origin_shift = 2 * np.pi * 6378137 / 2.0
    mx = df[lng_name] * origin_shift / 180.0
    my = np.log(np.tan((90 + df[lat_name]) * np.pi / 360.0)) / (np.pi / 180.0)
    my = my * origin_shift / 180.0
    mx = mx.apply(lambda x: np.around(x,0))
    my = my.apply(lambda y: np.around(y,0))
    df.loc[:,lng_name] = mx.astype(int)
    df.loc[:,lat_name] = my.astype(int)
    return df

In [76]:
projected_df = latlng_to_meters(sample_df,"LATITUDE","LONGITUDE")
source = ColumnDataSource(data=projected_df)

In [77]:
projected_df.head()

,GENTIME,LATITUDE,LONGITUDE,SPEED,HEADING,YAW_RATE,x,y
0,0.000000,5239042,-9322279,0.089478,203.625,0.000000,-0.000000,-0.000000
1,0.016661,5239049,-9322276,2.773806,203.625,11.610000,-0.006677,-0.015265
2,0.033328,5239051,-9322274,2.773806,203.625,16.709999,-0.013356,-0.030534
3,0.050004,5239052,-9322274,1.834291,203.625,16.740000,-0.020039,-0.045813
4,0.066660,5239052,-9322273,0.626343,203.625,1.950000,-0.026714,-0.061073


## Plotting

First we will plot the whole trip on custom map tiles.

In [101]:
lat_lng_carto = figure(plot_width = 500, 
                       plot_height = 500, 
                       title="Longitude/Latitude", 
                       match_aspect=True,
                       tools=[basic_tools])


lat_lng_carto.line(x='LONGITUDE', y ='LATITUDE',line_width=4, line_color="#40e0d0",source=source, name="lonlat_line")
lat_lng_carto.circle(x='LONGITUDE', y='LATITUDE', size=4, line_alpha = 0, fill_alpha=0,source=source, name="lonlat_circle")

latlng_hover_carto = HoverTool(names=["lonlat_line"], tooltips=[
                    ("Time","@GENTIME (min)"),
                    ("Longitude", "@LONGITUDE (meter)"),
                    ("Latitude", "@LATITUDE (meter)"),
                    ("Speed", "@SPEED (mph)")], 
                    point_policy = 'follow_mouse')

lat_lng_carto.add_tools(latlng_hover_carto)

lat_lng_carto.select(name="lonlat_line").nonselection_glyph = Line(line_alpha = 0)
lat_lng_carto.select(name="lonlat_line").selection_glyph = Line(line_alpha = 0)
lat_lng_carto.select(name="lonlat_circle").nonselection_glyph = Circle(fill_alpha = 0, line_color="#40e0d0", line_alpha=0.3)
lat_lng_carto.select(name="lonlat_circle").selection_glyph = Circle(fill_color = "#ff8c00", line_color="#ff8c00")

lat_lng_carto.axis.visible = False
lat_lng_carto.xgrid.grid_line_color = None
lat_lng_carto.ygrid.grid_line_color = None

lat_lng_carto.add_tile(CARTODBPOSITRON_RETINA)
show(lat_lng_carto)

Next is a line graph that shows speed over time.

In [89]:
speed_plot = figure(plot_width = 500, plot_height = 500, title="Speed vs Time", 
                    tools=[basic_tools])
speed_plot.line(x='GENTIME', y ='SPEED', line_width=2, source=source, name="speed_line", line_color="#40e0d0")
speed_plot.circle(x='GENTIME', y='SPEED', size=4, line_alpha = 0, fill_alpha=0, source=source, name="speed_circle")

speed_hover = HoverTool(names=["speed_line"], tooltips=[
                    ("Time", "$x min"),
                    ("Speed", "$y (mph)"),
                ], point_policy = 'follow_mouse')
                
speed_plot.add_tools(speed_hover)

speed_plot.select(name="speed_line").nonselection_glyph = Line(line_alpha = 0)
speed_plot.select(name="speed_line").selection_glyph = Line(line_alpha = 0)
speed_plot.select(name="spped_circle").nonselection_glyph = Circle(fill_alpha = 0,line_color="#40e0d0",line_alpha=0.3)
speed_plot.select(name="speed_circle").selection_glyph = Circle(fill_color = "#ff8c00", line_color="#ff8c00")

speed_plot.xaxis.axis_label = "Time (min)"
speed_plot.yaxis.axis_label = "Speed (mph)"
show(speed_plot)

The third plot shows yawrate(left or right turn) over time.

In [90]:
yaw_plot = figure(plot_width=500, plot_height=500, title="Time vs Yaw Rate",
                  tools=[basic_tools])
yaw_plot.line(x='YAW_RATE', y='GENTIME', line_width=2, source=source, name="yaw_line", line_color = "#40e0d0")
yaw_plot.circle(x='YAW_RATE', y='GENTIME', size=4, line_alpha = 0, fill_alpha=0, source=source, name="yaw_circle")

yaw_hover = HoverTool(names=["yaw_line"], tooltips=[
                    ("Time", "$y min"),
                    ("Yaw Rate", "$x (°/s)"),
                    ("Heading", "@HEADING°"),
                    ("Speed", "@SPEED (mph)")
                ], point_policy = 'follow_mouse')
                
yaw_plot.add_tools(yaw_hover)

yaw_plot.select(name="yaw_line").nonselection_glyph = Line(line_alpha = 0)
yaw_plot.select(name="yaw_line").selection_glyph = Line(line_alpha = 0)
yaw_plot.select(name="yaw_circle").nonselection_glyph = Circle(fill_alpha = 0, line_color="#40e0d0", line_alpha=0.3)
yaw_plot.select(name="yaw_circle").selection_glyph = Circle(fill_color = "#ff8c00", line_color="#ff8c00")

yaw_plot.xaxis.axis_label = "Yaw Rate (°/s)"
yaw_plot.yaxis.axis_label = "Time (min)"
show(yaw_plot)

The final graph is a radial plot that shows the vehicle's heading degree at each point.

In [83]:
tick_rng = (sample_df["GENTIME"].iloc[-1]-sample_df["GENTIME"].iloc[0])
tick_sep = min(radial_tick_scales, key = lambda x:abs(x-tick_rng/7))
num_ticks = int(tick_rng/tick_sep)+1
tick_bound = num_ticks*tick_sep

In [96]:
heading_plot = figure(plot_width=450, plot_height=500, title="Heading vs Time", tools=[basic_tools])

label_x = [(tick_bound), -0.15*tick_sep, -(tick_bound+tick_sep*1.1), -0.45*tick_sep]
label_y = [-0.25*tick_sep, (tick_bound), -0.25*tick_sep, -(tick_bound+0.55*tick_sep)]
labels = ["90°", "0°", "270°", "180°"]

for n in range(1,num_ticks+1):
    heading_plot.circle(x=0, y=0, radius=n*tick_sep, fill_alpha=0, line_alpha=1, line_color="#bbbbbb")
    label_x+=[n*tick_sep*np.cos(np.pi/4)+0.2*tick_sep]
    label_y+=[n*tick_sep*np.sin(np.pi/4)-0.2*tick_sep]
    labels+=[str(n*tick_sep)+" min"]
   
heading_plot.line(x=[-(tick_bound), (tick_bound)], y = [0, 0],  line_width=1.5, line_alpha=1, line_color="#bbbbbb")
heading_plot.line(y=[-(tick_bound), (tick_bound)], x = [0, 0],  line_width=1, line_alpha=1, line_color="#bbbbbb")
heading_plot.line(x = [0, tick_bound/1.41], y = [0, tick_bound/1.41],  line_width=1, line_alpha=1, line_color="#bbbbbb")


for n in range(0,24):   
    heading_plot.line(x = [(tick_bound-0.1*tick_sep)*np.cos(n*np.pi/12), (tick_bound+0.1*tick_sep)*np.cos(n*np.pi/12)],
                      y = [(tick_bound-0.1*tick_sep)*np.sin(n*np.pi/12), (tick_bound+0.1*tick_sep)*np.sin(n*np.pi/12)],
                        line_width = 1, line_color="#bbbbbb")


heading_plot.line(x='x', y='y', line_width=2, source=source, name="heading_line",line_color = "#40e0d0")
heading_plot.circle(x='x', y='y', size=4, line_alpha = 0, fill_alpha=0, source=source, name="heading_circle")
heading_plot.text(x = label_x, y = label_y, text = labels)


plt_range = sample_df["GENTIME"].iloc[-1]
heading_plot.x_range = Range1d(-plt_range,plt_range)
heading_plot.y_range = Range1d(-plt_range,plt_range)

heading_hover = HoverTool(names=["heading_line"], tooltips=[
                    ("Time", "@GENTIME min"),
                    ("Heading", "@HEADING°"),
                    ("Yaw Rate", "@YAW_RATE (°/s)"),
                    ("Speed", "@SPEED (mph)")
                ], point_policy = 'follow_mouse')

heading_plot.add_tools(heading_hover)

heading_plot.select(name="heading_line").nonselection_glyph = Line(line_alpha = 0)
heading_plot.select(name="heading_line").selection_glyph = Line(line_alpha = 0)
heading_plot.select(name="heading_circle").nonselection_glyph = Circle(fill_alpha = 0, line_color="#40e0d0", line_alpha=0.3)
heading_plot.select(name="heading_circle").selection_glyph = Circle(fill_color = "#ff8c00", line_color="#ff8c00")
heading_plot.x_range = Range1d(-(tick_bound+1.1*tick_sep), (tick_bound+1.1*tick_sep))
heading_plot.y_range = Range1d(-(tick_bound+1.1*tick_sep), (tick_bound+1.1*tick_sep))
heading_plot.xgrid.grid_line_color = None
heading_plot.ygrid.grid_line_color = None
heading_plot.outline_line_color = None
heading_plot.axis[0].axis_line_color = None
heading_plot.axis[1].axis_line_color = None
heading_plot.axis[0].ticker = FixedTicker()
heading_plot.axis[1].ticker = FixedTicker()
show(heading_plot)

Since we used the same ColumnDataSource object in all plots, we can arrange them in a grid which allows linked brushing.

In [102]:
p = gridplot([[lat_lng_carto, speed_plot], [heading_plot, yaw_plot]])
show(p)